## Parte 1: Validación de Datos
1.1 Análisis Exploratorio Inicial


In [0]:
%sql

SHOW TABLES IN workspace.operations;


##a) Conteo de registros por tabla

- ¿Cuántas transacciones hay en total? 

In [0]:
%sql
-- 1. Transacciones totales
SELECT COUNT(*) AS total_transacciones
FROM workspace.operations.gold_fact_sales_product;

- ¿Cuántos productos activos existen? 

In [0]:
%sql
-- 2. Productos activos
SELECT COUNT(*) AS productos_activos
FROM workspace.operations.gold_dim_product
WHERE STATUS = 'A';


- ¿Cuántas tiendas activas hay? 

In [0]:
%sql
-- 3. Tiendas activas
SELECT COUNT(*) AS tiendas_activas
FROM workspace.operations.gold_dim_organization
WHERE STORE_ACTIVE = 'Y';

- ¿Cuántos proveedores activos?


In [0]:
%sql
-- 4. Proveedores activos
SELECT COUNT(*) AS proveedores_activos
FROM workspace.operations.gold_dim_supplier
WHERE SUP_STATUS = 'A';


##b) Validación de rango de fechas

In [0]:
%sql
-- Rango de fechas en la tabla de ventas
SELECT 
    MIN(BUSINESS_DATE) AS fecha_minima,
    MAX(BUSINESS_DATE) AS fecha_maxima
FROM workspace.operations.gold_fact_sales_product;


## c) Validación de integridad referencial
- Se identifica cuántos registros en la tabla de hechos no tienen correspondencia en la tabla de dimensión asociada, usando LEFT JOIN y filtrando los registros donde la dimensión es NULL.
- Esto permite detectar violaciones de integridad referencial que podrían afectar análisis posteriores.


In [0]:
%sql
-- Verificar integridad SK_PRODUCT
SELECT COUNT(*) AS registros_sin_producto
FROM workspace.operations.gold_fact_sales_product f
LEFT JOIN workspace.operations.gold_dim_product p
ON f.SK_PRODUCT = p.SK_PRODUCT
WHERE p.SK_PRODUCT IS NULL;


In [0]:
%sql
-- Verificar integridad SK_ORGANIZATION
SELECT COUNT(*) AS registros_sin_tienda
FROM workspace.operations.gold_fact_sales_product f
LEFT JOIN workspace.operations.gold_dim_organization o
ON f.SK_ORGANIZATION = o.SK_ORGANIZATION
WHERE o.SK_ORGANIZATION IS NULL;

In [0]:
%sql
-- Verificar integridad SK_SUPPLIER
SELECT COUNT(*) AS registros_sin_proveedor
FROM workspace.operations.gold_fact_sales_product f
LEFT JOIN workspace.operations.gold_dim_supplier s
ON f.SK_SUPPLIER = s.SK_SUPPLIER
WHERE s.SK_SUPPLIER IS NULL;

In [0]:
%sql
-- Verificar integridad SK_CHANNEL
SELECT COUNT(*) AS registros_sin_canal
FROM workspace.operations.gold_fact_sales_product f
LEFT JOIN workspace.operations.gold_dim_channel c
ON f.SK_CHANNEL = c.SK_CHANNEL
WHERE c.SK_CHANNEL IS NULL;


In [0]:
%sql
-- Verificar integridad SK_CURRENCY
SELECT COUNT(*) AS registros_sin_moneda
FROM workspace.operations.gold_fact_sales_product f
LEFT JOIN workspace.operations.gold_dim_currency cur
ON f.SK_CURRENCY = cur.SK_CURRENCY
WHERE cur.SK_CURRENCY IS NULL;

## d) Validación de valores nulos y negativos


- Ventas nulas
- Ventas negativas
- Unidades nulas o negativas

In [0]:
%sql
-- Ventas nulas
SELECT COUNT(*) AS ventas_nulas
FROM workspace.operations.gold_fact_sales_product
WHERE TOTAL_SALE IS NULL;

In [0]:
%sql
-- Ventas negativas
SELECT COUNT(*) AS ventas_negativas
FROM workspace.operations.gold_fact_sales_product
WHERE TOTAL_SALE < 0;

In [0]:
%sql
-- Unidades inválidas
SELECT COUNT(*) AS unidades_invalidas
FROM workspace.operations.gold_fact_sales_product
WHERE UNITS IS NULL OR UNITS <= 0;


In [0]:
%sql
-- Validamos tambien que las unidades sean numeros enteros
SELECT *
FROM workspace.operations.gold_fact_sales_product
WHERE UNITS % 1 <> 0

## e) Validación de consistencia de cálculos



In [0]:
%sql
-- Validar TOTAL_SALE + TAX = TOTAL_SALE_TAX
SELECT COUNT(*) AS inconsistencias_total
FROM workspace.operations.gold_fact_sales_product
WHERE TOTAL_SALE + TAX <> TOTAL_SALE_TAX;


##Validaciones adicionales

1. **Detecta márgenes negativos (costo > total), señalando posibles inconsistencias en el registro o transacciones a pérdida.**

In [0]:
%sql
SELECT COUNT(*) 
FROM workspace.operations.gold_fact_sales_product
WHERE TOTAL_SALE - TOTAL_COST < 0;


**2. Valida que el impuesto (IVA 16%) guarde la proporción correcta respecto al total de la venta para asegurar la integridad de los datos**

In [0]:
%sql
SELECT COUNT(*) 
FROM workspace.operations.gold_fact_sales_product
WHERE ROUND(TAX,2) <> ROUND(TOTAL_SALE*0.16,2);



**3. Validación de que Q_TRANS siempre fuese 1**

In [0]:
%sql
SELECT count(*)
FROM workspace.operations.gold_fact_sales_product
WHERE Q_TRANS <> 1;



**4. Detección y validación de registros duplicados**


In [0]:
# -----------------------------
# Validación de duplicados
# -----------------------------
from pyspark.sql import functions as F

# Lista de tablas a validar
# Estas son las tablas que contienen dimensiones y hechos; queremos asegurarnos de que no existan registros duplicados en ninguna.
tablas = [
    "gold_dim_product",
    "gold_dim_organization",
    "gold_dim_supplier",
    "gold_dim_channel",
    "gold_dim_currency",
    "gold_fact_sales_product"
]

# Iterar sobre cada tabla para revisar duplicados
for tabla in tablas:
    # Cargar la tabla en un DataFrame de Spark
    df = spark.table(f"workspace.operations.{tabla}")
    
    # Contar el total de registros en la tabla
    total = df.count()
    
    # Contar el número de registros distintos
    # Lógica: df.distinct() elimina cualquier fila que esté completamente repetida en todas las columnas
    total_distinct = df.distinct().count()
    
    # Mostrar resultados
    print(f"\nTabla: {tabla}")
    print(f"Total registros: {total}, Total distintos: {total_distinct}")
    
    # Comparar total vs total_distinct
    # Si total == total_distinct, no hay duplicados; si es mayor, indica cuántos registros se repiten exactamente
    if total == total_distinct:
        print("✅ No hay duplicados")
    else:
        print(f"⚠️ Hay duplicados! Diferencia: {total - total_distinct} registros")


**5. Análisis de distribución y detección de outliers en la tabla de ventas (VES y USD)**


**5.1 Gráficos para ventas en VES**

In [0]:
# Importar librerías
import pandas as pd
import matplotlib.pyplot as plt

# Leer la tabla de Databricks
df = spark.table("workspace.operations.gold_fact_sales_product")

# Filtrar solo transacciones en VES
df = df.join(
    spark.table("workspace.operations.gold_dim_currency"),
    on="SK_CURRENCY",
    how="inner"
).filter("CURRENCY_ABR = 'VES'").toPandas()

# Histograma de ventas totales en VES
plt.figure(figsize=(10,6))
plt.hist(df['TOTAL_SALE'], bins=50, color='skyblue', edgecolor='black')
plt.title("Distribución de TOTAL_SALE en VES")
plt.xlabel("TOTAL_SALE (VES)")
plt.ylabel("Cantidad de transacciones")
plt.show()

# Boxplot para ver outliers en VES
plt.figure(figsize=(8,6))
plt.boxplot(df['TOTAL_SALE'], vert=False)
plt.title("Boxplot de TOTAL_SALE en VES")
plt.xlabel("TOTAL_SALE (VES)")
plt.show()


**5.2 Gráficos para ventas en USD**

In [0]:
# Importar librerías
import pandas as pd
import matplotlib.pyplot as plt

# Leer la tabla de Databricks
df = spark.table("workspace.operations.gold_fact_sales_product")

# Filtrar solo transacciones en USD
df = df.join(
    spark.table("workspace.operations.gold_dim_currency"),
    on="SK_CURRENCY",
    how="inner"
).filter("CURRENCY_ABR = 'USD'").toPandas()

# Histograma de ventas totales en USD
plt.figure(figsize=(10,6))
plt.hist(df['TOTAL_SALE'], bins=50, color='lightgreen', edgecolor='black')
plt.title("Distribución de TOTAL_SALE en USD")
plt.xlabel("TOTAL_SALE (USD)")
plt.ylabel("Cantidad de transacciones")
plt.show()

# Boxplot para ver outliers en USD
plt.figure(figsize=(8,6))
plt.boxplot(df['TOTAL_SALE'], vert=False)
plt.title("Boxplot de TOTAL_SALE en USD")
plt.xlabel("TOTAL_SALE (USD)")
plt.show()


**5.3 Cantidad de transacciones por moneda**

In [0]:
# Importar librerías
import pandas as pd
import matplotlib.pyplot as plt

# Leer la tabla de Databricks
df = spark.table("workspace.operations.gold_fact_sales_product")

# Unir con la tabla de monedas para obtener CURRENCY_ABR
df = df.join(
    spark.table("workspace.operations.gold_dim_currency"),
    on="SK_CURRENCY",
    how="inner"
)

# Agrupar por moneda y contar transacciones
transacciones_por_moneda = df.groupBy("CURRENCY_ABR").count().toPandas()

# Gráfico de barras
plt.figure(figsize=(8,6))
plt.bar(transacciones_por_moneda['CURRENCY_ABR'], transacciones_por_moneda['count'],
        color=['skyblue', 'lightgreen'], edgecolor='black')
plt.title("Cantidad de transacciones por moneda")
plt.xlabel("Moneda")
plt.ylabel("Cantidad de transacciones")
plt.show()


**6. Validaciones de valores esperados en tablas dimensionales**

**6.1 Validaciones en la tabla producto**
- Se valida que el status sea A o I

In [0]:
%sql
SELECT STATUS, COUNT(*) AS cantidad
FROM workspace.operations.gold_dim_product
GROUP BY STATUS;


**6.2 Validaciones en la tabla organización**
- Se valida que el nombre sea Capital, Occidente, Centro o Oriente

In [0]:
%sql
-- Validar REGION_NAME
SELECT REGION_NAME, COUNT(*) AS cantidad
FROM workspace.operations.gold_dim_organization
GROUP BY REGION_NAME;



- Se valida que el formato de la tienda sea Express, Plus, Super o Mega

In [0]:
%sql
-- Validar FORMAT_NAME
SELECT FORMAT_NAME, COUNT(*) AS cantidad
FROM workspace.operations.gold_dim_organization
GROUP BY FORMAT_NAME;

- Se valida que el estado de la tienda sea Y o N

In [0]:
%sql
-- Validar STORE_ACTIVE
SELECT STORE_ACTIVE, COUNT(*) AS cantidad
FROM workspace.operations.gold_dim_organization
GROUP BY STORE_ACTIVE;

**6.3 Validaciones en la tabla supplier**
- Se valida que el status sea A o I

In [0]:
%sql
-- Validar valores de SUP_STATUS
SELECT SUP_STATUS, COUNT(*) AS cantidad
FROM workspace.operations.gold_dim_supplier
GROUP BY SUP_STATUS;


- Se valida que el nombre sea Tienda Física, E-Commerce, App Móvil o Call Center

In [0]:
%sql
-- Validar CHANNEL_NAME
SELECT CHANNEL_NAME, COUNT(*) AS cantidad
FROM workspace.operations.gold_dim_channel
GROUP BY CHANNEL_NAME;

- Se valida que el tipo de canal sea Retail, Online, Telefónico

In [0]:
%sql
-- Validar CHANNEL_TYPE_NAME
SELECT CHANNEL_TYPE_NAME, COUNT(*) AS cantidad
FROM workspace.operations.gold_dim_channel
GROUP BY CHANNEL_TYPE_NAME;

**6.4 Validaciones en la tabla currency**
- Se valida que sea VES o USD

In [0]:
%sql
-- Validar CURRENCY_ABR
SELECT CURRENCY_ABR, COUNT(*) AS cantidad
FROM workspace.operations.gold_dim_currency
GROUP BY CURRENCY_ABR;

**7. Validacion de valores nulos en las tablas**

In [0]:
from pyspark.sql import functions as F

# -------------------------------------------------
# Validación de valores nulos en columnas críticas
# -------------------------------------------------

# Diccionario de tablas y columnas críticas
tablas = {
    "gold_dim_product": ["SK_PRODUCT", "ITEM", "ITEM_DESC", "CLASS_NAME", "DEPT_NAME", "BRAND_NAME", "STATUS"],
    "gold_dim_organization": ["SK_ORGANIZATION", "STORE_ID", "STORE_NAME", "CITY", "REGION_NAME", "FORMAT_NAME", "STORE_ACTIVE"],
    "gold_dim_supplier": ["SK_SUPPLIER", "SUPPLIER", "SUP_NAME", "SUP_STATUS"],
    "gold_dim_channel": ["SK_CHANNEL", "CHANNEL_NAME", "CHANNEL_TYPE_NAME"],
    "gold_dim_currency": ["SK_CURRENCY", "CURRENCY_ABR", "CURRENCY_NAME"],
    "gold_fact_sales_product": ["BUSINESS_DATE", "SK_PRODUCT", "SK_ORGANIZATION", "SK_SUPPLIER", 
                                "SK_CHANNEL", "SK_CURRENCY", "TOTAL_SALE", "TOTAL_SALE_TAX", "TAX", 
                                "TOTAL_COST", "UNITS", "Q_TRANS"]
}

# Iterar sobre cada tabla para validar nulos
for tabla, columnas in tablas.items():
    print(f"\n--- Tabla: {tabla} ---")
    df = spark.table(f"workspace.operations.{tabla}")
    
    # Validación de nulos por columna crítica
    nulos = df.select([F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in columnas])
    
    # Mostrar resultados: cantidad de nulos por columna
    nulos.show(truncate=False)


##1.2 Documento de Hallazgos


%md
## Resumen de Validación de Datos

✅ Validaciones que pasaron exitosamente  
⚠️ Problemas encontrados  
📋 Recomendaciones para mejorar la calidad de datos  

---

### a) Conteo de registros por tabla ✅
- **Transacciones totales:** 432,987  
- **Productos activos:** 749  
- **Tiendas activas:** 94  
- **Proveedores activos:** 7  

### b) Validación de rango de fechas ✅
- Todas las transacciones están dentro del rango **julio - septiembre 2025**.  

### c) Validación de integridad referencial ✅
- Todas las claves foráneas (`SK_PRODUCT`, `SK_ORGANIZATION`, `SK_SUPPLIER`, `SK_CHANNEL`, `SK_CURRENCY`) tienen correspondencia en sus tablas de dimensiones.  
- **Resultado:** 0 registros sin correspondencia  
> Esto indica que **no hay registros huérfanos** en la tabla de ventas.

### d) Validación de valores nulos y negativos ✅
- **Ventas nulas:** 0 → No se encontraron ventas nulas  
- **Ventas negativas:** 0 → No se encontraron ventas negativas  
- **Unidades inválidas (nulas o <=0):** 0 → No se encontraron unidades inválidas o nulas  

### e) Validación de consistencia de cálculos ⚠️
- **TOTAL_SALE + TAX = TOTAL_SALE_TAX**  
  - Se encontraron **95,687 registros** que no cumplen esta relación.  
- **TAX = 16% de TOTAL_SALE**  
  - Ningún registro coincide exactamente con la regla del 16%.  

> 📋 Recomendación:  
- Mantener los **datos crudos** tal como se reciben.  
- Realizar **cálculos de impuestos y totales en los procesos ETL**, para garantizar consistencia y evitar errores manuales.

---

### f) Validaciones adicionales realizadas ✅
- Se verificaron los campos con **valores predefinidos** en todas las tablas dimensionales como por ejemplo:  
  - `STATUS` en productos: solo `A` o `I`   
  - `CHANNEL_TYPE_NAME` en canales: solo `Retail`, `Online`, `Telefónico`  
  - `CURRENCY_ABR` en monedas: solo `VES` o `USD`  
- Todos los registros coincidieron con los valores esperados; **no se encontraron inconsistencias**.  

- Se validó que no existieran **transacciones de pérdida o duplicadas**; no se encontraron registros duplicados en ninguna tabla.  

- Se realizó un análisis de **distribución de TOTAL_SALE** en ambas monedas (VES y USD):  
  - La mayoría de las ventas son pequeñas (< 800), pero existen registros atípicos con valores mucho más altos (~1,700–2,100), representando **outliers**.  
  -El comportamiento idéntico de VES y USD es **improbable en un escenario real**, lo que normalmente podría indicar **falta de conversión cambiaria o error en la categorización de moneda en la fuente de datos**.  
     **Adicional:** se concluyó que hubo **más transacciones en VES que en USD**, lo cual es consistente con el comportamiento esperado de ventas locales en retail venezolano.

---

## ✅ Conclusión General
Los datos del trimestre presentan buena calidad en cuanto a:  
- Conteo de registros  
- Fechas  
- Integridad referencial  
- Valores nulos y negativos  
- Consistencia de campos con valores predefinidos  
- Ausencia de duplicados  

El principal punto a mejorar son los **cálculos derivados** (impuesto y total con impuestos), que deben automatizarse para garantizar métricas consistentes en el análisis de negocio.  

